In [1]:
!pip install cirq

In [1]:
import cirq
import math
from cirq.contrib.svg import SVGCircuit
import sympy
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from tqdm import tqdm
from numpy import genfromtxt

In [2]:
## DEFINITION
N = 8
D = 3
A = 3
A = 0.3
##sigma=np.zeros((N,N))
# sigma = np.array([[9.980e-05, 4.250e-05, 3.720e-05, 4.030e-05],
#                   [4.250e-05, 1.005e-04, 4.110e-05, 1.520e-05],
#                   [3.720e-05, 4.110e-05, 1.813e-04, 1.790e-05],
#                   [4.030e-05, 1.520e-05, 1.790e-05, 2.531e-04]])
# mu = np.array([4.01e-04,  6.10e-05,  9.16e-04, -6.19e-04])
sigma = genfromtxt('Sigma.csv', delimiter=',')
mu = np.array([ 0.27639461,  0.35676064,  0.30267724, -0.0343477,   0.05617082, -0.05095225,
 -0.03738396,  0.83449729])
y = np.zeros(N)
qubits = [cirq.LineQubit(i) for i in range(2*N)]
L=len(qubits)
index_grid_pair = [(2*i,2*i+1) for i in range(0,N) ]

lam=0.9

In [3]:
## INITIALIZATION AND MEASURE
def circuit_hard():
    initial_hard = cirq.Circuit()
    # Initialising the first D = N/2 qubits in |10> state (s+ being the first qubit and s- being the 2nd)
    for i in range(int(D)):
            initial_hard.append(cirq.X(qubits[index_grid_pair[i][0]]))
    # Initialising the remaining N-D qubits in the |phi_00> state
    for i in range(int(D), N):
                    initial_hard.append(cirq.H(qubits[index_grid_pair[i][0]]))
                    initial_hard.append(cirq.CNOT(qubits[index_grid_pair[i][0]], qubits[index_grid_pair[i][1]]))
    return initial_hard

def circuit_hadamard():
    hadamard=cirq.Circuit()
    for i in range(L):
        hadamard.append(cirq.H(qubits[i]))
    return hadamard

def circuit_measurements():
    measurements=cirq.Circuit()
    for i in range(L):
        measurements.append(cirq.measure(qubits[i]))
    return measurements

## MIXER OPERATOR
def circuit_mixer(beta):
    mixer=cirq.Circuit()
    for i in range(L):
        mixer.append(cirq.rx(2*beta).on(qubits[i]))
    return mixer

def circuit_mixer_hard(beta):
    mixer_hard = cirq.Circuit()
    # Even parity mixer bands Ref: Fig-1 of paper
    for i in range(0,2*N-2,2):
        mixer_hard.append(cirq.ISwapPowGate(exponent=(4.0*(-beta)/np.pi)).on(qubits[i],qubits[i+2])) 
    # Odd parity mixer bands Ref: Fig-1 of paper
    for i in range(1,2*N-2,2):
        mixer_hard.append(cirq.ISwapPowGate(exponent=(4.0*(-beta)/np.pi)).on(qubits[i],qubits[i+2]))
    # Long position parity mixer Ref: Fig-1 of paper
    mixer_hard.append(cirq.ISwapPowGate(exponent=(4.0*(-beta)/np.pi)).on(qubits[2*N-1],qubits[1]))
    # Short position parity mixer Ref: Fig-1 of paper
    mixer_hard.append(cirq.ISwapPowGate(exponent=(4.0*(-beta)/np.pi)).on(qubits[2*N-2],qubits[0]))
    return mixer_hard

## RETURNS OPERATOR
def circuit_returns(gamma):
    returns=cirq.Circuit()
    for i in range(L):
        returns.append(cirq.rz((-1)**int(i)*(1-lam)*gamma*mu[int(i/2)]).on(qubits[i]))
    return returns

## RISK OPERATOR
def circuit_risk(gamma):
    combinations=[]
    for i in range(L):
        for j in range(L):
            if i!=j:
                combinations.append((i,j))
    numcombinations=len(combinations)
    risk=cirq.Circuit()
    for i in range(numcombinations):
        q1=combinations[i][0]
        q2=combinations[i][1]
        ii=math.floor(q1/2)
        jj=math.floor(q2/2)
        sign=-(-1)**((q1 % 2)+(q2 %2))
        risk.append(cirq.ZZPowGate(exponent= 1/2*sign*gamma*lam*sigma[ii][jj]).on(qubits[q1],qubits[q2]))
    return risk

##  CONSTRAINT OPERATOR
def circuit_constraint(gamma):
    combinations=[]
    for i in range(L):
        for j in range(L):
            if i!=j:
                combinations.append((i,j))
    numcombinations=len(combinations)
    constraint=cirq.Circuit()
    for i in range(len(qubits)):
        sign=(-1)**i
        constraint.append(cirq.rz( sign *2 *gamma*A*D).on(qubits[i]))
    for i in range(numcombinations):
        q1=combinations[i][0]
        q2=combinations[i][1]
        sign=-(-1)**((q1 % 2)+(q2 %2))
        constraint.append(cirq.ZZPowGate(exponent= 1/2*sign*A*gamma).on(qubits[q1],qubits[q2]))
    return constraint

## CIRCUIT BUILDER
def circuit_builder(beta,gamma):
    hadamard=circuit_hadamard()
    returns=circuit_returns(gamma)
    risk=circuit_risk(gamma)
    constraint=circuit_constraint(gamma)
    mixer=circuit_mixer(beta)
    measurements=circuit_measurements()
    return cirq.Circuit(hadamard, returns, risk, constraint, mixer, measurements)

def circuit_builder_p(beta,gamma): #beta and gamma are arrays of length
    if len(beta)==len(gamma):
        p=len(beta)
    else:
        print("Error: beta and gamma must have the same length")
        return 1;
    circuit=cirq.Circuit()
    circuit.append(circuit_hadamard())
    for i in range(p):
        circuit.append(circuit_returns(gamma[i]))
        circuit.append(circuit_risk(gamma[i]))
        circuit.append(circuit_constraint(gamma[i]))
        circuit.append(circuit_mixer(beta[i]))
    circuit.append(circuit_measurements())
    return circuit

def circuit_builder_p_hard(beta,gamma): #Creates a circuit of depth p (beta and gamma must be arrays of length p)
    if len(beta)==len(gamma):
        p=len(beta)
    else:
        print("Error: beta and gamma must have the same length")
        return 1;
    circuit=cirq.Circuit()
    circuit.append(circuit_hard())
    for i in range(p):
        circuit.append(circuit_returns(gamma[i]))
        circuit.append(circuit_risk(gamma[i]))
        circuit.append(circuit_mixer_hard(beta[i]))
    circuit.append(circuit_measurements())
    return circuit

## COST FUNCTION
def cost_function(z):
    return lam*np.dot(z,np.dot(sigma,z))+A*(D-sum(z))**2-(1-lam)*np.dot(mu,z)

def cost_function_hard(z):
    return lam*np.dot(z,np.dot(sigma,z))-(1-lam)*np.dot(mu,z)

def cost_function_angle(beta,gamma,numsim):
    simulator = cirq.Simulator()
    Cost_min=100
    circuit=circuit_builder_p(beta,gamma)
    for i in range(numsim):
        result = simulator.run(circuit)
        list=[]
        z=[]
        for value in result.measurements.values():
            list.append(value[0,0])
        for j in range(N):
            z.append((list[2*j]-list[2*j+1]))
        C=cost_function(z)
        if C<Cost_min:
            Cost_min=C
            positions=z
    return positions, Cost_min

def cost_function_angle_hard(beta,gamma,numsim):
    simulator = cirq.Simulator()
    Cost_min=100
    circuit=circuit_builder_p_hard(beta,gamma)
    for i in range(numsim):
        result = simulator.run(circuit)
        list=[]
        z=[]
        for value in result.measurements.values():
            list.append(value[0,0])
        for j in range(N):
            z.append((list[2*j]-list[2*j+1]))
        C=cost_function_hard(z)
        if C<Cost_min:
            Cost_min=C
            positions=z
    return positions, Cost_min

def cost_function_angle_mean(beta,gamma,numsim):
    simulator = cirq.Simulator()
    circuit=circuit_builder_p(beta,gamma)
    C=[]
    for i in range(numsim):
        result = simulator.run(circuit)
        list=[]
        z=[]
        for value in result.measurements.values():
            list.append(value[0,0])
        for j in range(N):
            z.append((list[2*j]-list[2*j+1]))
        C.append(cost_function(z))
    return np.mean(C)

def cost_function_angle_mean_hard(beta,gamma,numsim):
    simulator = cirq.Simulator()
    circuit=circuit_builder_p_hard(beta,gamma)
    C=[]
    for i in range(numsim):
        result = simulator.run(circuit)
        list=[]
        z=[]
        for value in result.measurements.values():
            list.append(value[0,0])
        for j in range(N):
            z.append((list[2*j]-list[2*j+1]))
        C.append(cost_function_hard(z))
    return np.mean(C)

## GRID SEARCH
def grid_search(numpoints, numsim):
    Cost_min=100
    simulator = cirq.Simulator()
    positions=[]
    betas= np.linspace(0, 2 * np.pi, numpoints)
    gammas= np.linspace(0, 2 * np.pi, numpoints)
    vals = []
    for beta in betas:
        for gamma in gammas:
            circuit=circuit_builder(beta,gamma)
            for i in range(numsim):
                result = simulator.run(circuit)
                list=[]
                z=[]
                for value in result.measurements.values():
                    list.append(value[0,0])
                for i in range(N):
                    z.append((list[2*i]-list[2*i+1]))
                C=cost_function(z)
                vals.append(C)
                if C<Cost_min:
                    Cost_min=C
                    positions=z
                    
    fig1, ax1 = plt.figure(1)
    plot = ax1.imshow(z,interpolation='none',cmap=p.cm.jet,origin='lower')  
    fig1.colorbar(plot, ax=ax1)
    ax1.set_xlabel('label in x')
    ax1.set_ylabel('label in y')
    ax1.set_title('title')
    plt.show()
    
    return positions, Cost_min

def grid_search_2(numpoints, numsim):
    Cost_min=100
    simulator = cirq.Simulator()
    positions=[]
    betas= np.linspace(0, 2 * np.pi, numpoints)
    gammas= np.linspace(0, 2 * np.pi, numpoints)
    for beta1 in betas:
        for gamma1 in gammas:
            for beta2 in betas:
                for gamma2 in gammas:
                    circuit=circuit_builder_p([beta1,beta2],[gamma1,gamma2])
                    for i in range(numsim):
                        result = simulator.run(circuit)
                        list=[]
                        z=[]
                        for value in result.measurements.values():
                            list.append(value[0,0])
                        for i in range(N):
                            z.append((list[2*i]-list[2*i+1]))
                        C = cost_function(z)
                        if C < Cost_min:
                            Cost_min = C
                            positions = z
    return positions, Cost_min

def grid_search_2_hard(numpoints, numsim):
    Cost_min=100
    simulator = cirq.Simulator()
    positions=[]
    betas= np.linspace(0, 2 * np.pi, numpoints)
    gammas= np.linspace(0, 2 * np.pi, numpoints)
    for beta1 in betas:
        for gamma1 in gammas:
            for beta2 in betas:
                for gamma2 in gammas:
                    circuit=circuit_builder_p_hard([beta1,beta2],[gamma1,gamma2])
                    for i in range(numsim):
                        result = simulator.run(circuit)
                        list=[]
                        z=[]
                        for value in result.measurements.values():
                            list.append(value[0,0])
                        for i in range(N):
                            z.append((list[2*i]-list[2*i+1]))
                        C = cost_function_hard(z)
                        if C < Cost_min:
                            Cost_min = C
                            positions = z
    return positions, Cost_min

def cross_entropy_mean(p,iterations,Nce_samples,f_elite):
    width = np.identity(2*p)
    centre = np.random.uniform(0.0,2*np.pi,2*p)
    print("MEAN")
    for i in range(iterations):
        # Generate N-samples from the multivariate Gaussian
        X = np.random.multivariate_normal(centre,width,Nce_samples)
        E = np.zeros(len(X))
        data = []
        
        for k in range(len(X)):
            E[k] = cost_function_angle_mean(X[k][0:p],X[k][p:],20)
            data.append([*X[k],E[k]])

        # Sort the value according to the best
        sorted_data = np.array(sorted(data, key=lambda x: x[2*p], reverse=False))

        # Now compute the new averages
        N_elite = int(f_elite*Nce_samples)
        X_elite = sorted_data[0:N_elite,0:2*p]
        E_elite = sorted_data[0:N_elite,2*p:]
        
        print('k={}, mean_E={:.6f}, std_E={:.6f}, beta={}, gamma={}'.format(i,E_elite.mean(),E_elite.std(),centre[0:p],centre[p:]))

        # Compute the new values of mu and sigma
        centre = np.mean(X_elite,axis=0)
        width = np.cov(X_elite.T)

    betas = centre[0:p]
    gammas = centre[p:]
    return betas, gammas

def cross_entropy_mean_hard(p,iterations,Nce_samples,f_elite):
    width = np.identity(2*p)
    centre = np.random.uniform(0.0,2*np.pi,2*p)
    print("MEAN")
    for i in range(iterations):
        # Generate N-samples from the multivariate Gaussian
        X = np.random.multivariate_normal(centre,width,Nce_samples)
        E = np.zeros(len(X))
        data = []
        
        for k in range(len(X)):
            E[k] = cost_function_angle_mean_hard(X[k][0:p],X[k][p:],20)
            data.append([*X[k],E[k]])

        # Sort the value according to the best
        sorted_data = np.array(sorted(data, key=lambda x: x[2*p], reverse=False))

        # Now compute the new averages
        N_elite = int(f_elite*Nce_samples)
        X_elite = sorted_data[0:N_elite,0:2*p]
        E_elite = sorted_data[0:N_elite,2*p:]
        
        print('k={}, mean_E={:.6f}, std_E={:.6f}, beta={}, gamma={}'.format(i,E_elite.mean(),E_elite.std(),centre[0:p],centre[p:]))

        # Compute the new values of mu and sigma
        centre = np.mean(X_elite,axis=0)
        width = np.cov(X_elite.T)


    # get the other results
    betas = centre[0:p]
    gammas = centre[p:]
    return betas, gammas
    
def cross_entropy_min(p,iterations,Nce_samples,f_elite):
    width = np.identity(2*p)
    centre = np.random.uniform(0.0,2*np.pi,2*p)
    print("MIN")
    for i in range(iterations):
        # Generate N-samples from the multivariate Gaussian
        X = np.random.multivariate_normal(centre,width,Nce_samples)
        E = np.zeros(len(X))
        data = []
        
        for k in range(len(X)):
            E[k] = cost_function_angle(X[k][0:p],X[k][p:],10)[1]
            data.append([*X[k],E[k]])

        # Sort the value according to the best
        sorted_data = np.array(sorted(data, key=lambda x: x[2*p], reverse=False))

        # Now compute the new averages
        N_elite = int(f_elite*Nce_samples)
        X_elite = sorted_data[0:N_elite,0:2*p]
        E_elite = sorted_data[0:N_elite,2*p:]
        
        print('k={}, mean_E={:.6f}, std_E={:.6f}, beta={}, gamma={}'.format(i,E_elite.mean(),E_elite.std(),centre[0:p],centre[p:]))

        # Compute the new values of mu and sigma
        centre = np.mean(X_elite,axis=0)
        width = np.cov(X_elite.T)

    betas = centre[0:p]
    gammas = centre[p:]
    return betas, gammas

def cross_entropy_min_hard(p,iterations,Nce_samples,f_elite):
    width = np.identity(2*p)
    centre = np.random.uniform(0.0,2*np.pi,2*p)
    print("MIN")
    for i in range(iterations):
        # Generate N-samples from the multivariate Gaussian
        X = np.random.multivariate_normal(centre,width,Nce_samples)
        E = np.zeros(len(X))
        data = []
        
        for k in range(len(X)):
            E[k] = cost_function_angle_hard(X[k][0:p],X[k][p:],10)[1]
            data.append([*X[k],E[k]])

        # Sort the value according to the best
        sorted_data = np.array(sorted(data, key=lambda x: x[2*p], reverse=False))

        # Now compute the new averages
        N_elite = int(f_elite*Nce_samples)
        X_elite = sorted_data[0:N_elite,0:2*p]
        E_elite = sorted_data[0:N_elite,2*p:]
        
        print('k={}, mean_E={:.6f}, std_E={:.6f}, beta={}, gamma={}'.format(i,E_elite.mean(),E_elite.std(),centre[0:p],centre[p:]))

        # Compute the new values of mu and sigma
        centre = np.mean(X_elite,axis=0)
        width = np.cov(X_elite.T)

    # get the other results
    betas = centre[0:p]
    gammas = centre[p:]

    return betas,gammas

def quality_parameters(beta,gamma,E_treshold,R):
    simulator = cirq.Simulator()
    p=0
    q=0
    circuit=circuit_builder_p(beta,gamma)
    for i in range(R):
        result = simulator.run(circuit)
        list=[]
        z=[]
        for value in result.measurements.values():
            list.append(value[0,0])
        for i in range(N):
            z.append((list[2*i]-list[2*i+1]))
        C=cost_function(z)
        if(C<E_treshold):
            p+=1
        if(z==[1,    -1,     1,     1,     0,     0,     1,     0]):
            q+=1
    return p/R,q/R,cost_function_angle_mean(beta,gamma,R)

def quality_parameters_hard(beta,gamma,E_treshold,R):
    simulator = cirq.Simulator()
    p=0
    q=0
    circuit=circuit_builder_p_hard(beta,gamma)
    for i in range(R):
        result = simulator.run(circuit)
        list=[]
        z=[]
        for value in result.measurements.values():
            list.append(value[0,0])
        for i in range(N):
            z.append((list[2*i]-list[2*i+1]))
        C=cost_function_hard(z)
        if(C<E_treshold):
            p+=1
        if(z==[1,    -1,     1,     1,     0,     0,     1,     0]):
            q+=1
    return p/R,q/R,cost_function_angle_mean_hard(beta,gamma,R)

In [5]:
grid_search(5,5)

TypeError: cannot unpack non-iterable Figure object

<Figure size 432x288 with 0 Axes>

In [7]:

p=2   #Circuit depth # 1,2,3
iterations=10; # Number of iterations for cross entropy 1,5,10,20
Nce_samples=10; # Number of samples 10,15,20
f_elite=0.2;   # fraction of best samples 0.2,0.4, 1

Num=10# Number of times we run the loop 
E_treshold=0.005; # Maximum energy which we tolerate
R=1000; # Number of times we average the quality parameters
beta_min=np.zeros((Num,p)) #Circuit parameters beta and gamma for min strategy
gamma_min=np.zeros((Num,p))
beta_mean=np.zeros((Num,p)) #Circuit parameters beta and gamma for mean strategy
gamma_mean=np.zeros((Num,p))
Q_min=np.zeros((Num,3))    # Quality parameters for min strategy
Q_mean=np.zeros((Num,3))   # Quality parameters for mean strategy

for ii in range(Num):
    beta_min[ii],gamma_min[ii]=cross_entropy_min(p,iterations,Nce_samples,f_elite)
    beta_mean[ii],gamma_mean[ii]=cross_entropy_mean(p,iterations,Nce_samples,f_elite)
    Q_min[ii]=quality_parameters(beta_min[ii],gamma_min[ii],E_treshold,R)
    Q_mean[ii]=quality_parameters(beta_mean[ii],gamma_mean[ii],E_treshold,R)
    
beta_min,gamma_min,Q_min, Q_mean

MIN
k=0, mean_E=0.615708, std_E=0.326502, beta=[2.09329011 2.82762247], gamma=[1.65551684 0.73736621]
k=1, mean_E=0.242109, std_E=0.202059, beta=[3.23520717 1.52551151], gamma=[2.85529211 0.5931836 ]
k=2, mean_E=0.554907, std_E=0.035319, beta=[3.19363524 1.64216193], gamma=[2.91113281 0.59014101]
k=3, mean_E=0.274413, std_E=0.237380, beta=[3.0573736  2.02451065], gamma=[3.09416362 0.58016822]
k=4, mean_E=0.375841, std_E=0.008333, beta=[3.21351648 1.58637539], gamma=[2.88442773 0.59159609]
k=5, mean_E=0.232685, std_E=0.045523, beta=[3.20104458 1.62137141], gamma=[2.90118037 0.59068329]
k=6, mean_E=0.353150, std_E=0.014358, beta=[3.18167697 1.6757167 ], gamma=[2.92719553 0.5892658 ]
k=7, mean_E=0.347072, std_E=0.210266, beta=[3.19054665 1.65082848], gamma=[2.91528151 0.58991496]
k=8, mean_E=0.443534, std_E=0.306727, beta=[3.18978451 1.65296704], gamma=[2.91630523 0.58985918]
k=9, mean_E=0.398206, std_E=0.115148, beta=[3.18949708 1.65377357], gamma=[2.91669132 0.58983814]
MEAN
k=0, mean_E

KeyboardInterrupt: 

In [ ]:
beta_min_p1_iter10_Nce10_f02_soft=beta_min
gamma_min_p1_iter10_Nce10_f02_soft=gamma_min
beta_mean_p1_iter10_Nce10_f02_soft=beta_mean
gamma_mean_p1_iter10_Nce10_f02_soft=gamma_mean
Q_min_p1_iter10_Nce10_f02_soft=Q_min
Q_mean_p1_iter10_Nce10_f02_soft=Q_mean
np.savetxt('beta_min_p1_iter10_Nce10_f02_soft.csv', beta_min_p1_iter10_Nce10_f02_soft, delimiter=',')
np.savetxt('beta_mean_p1_iter10_Nce10_f02_soft.csv', beta_mean_p1_iter10_Nce10_f02_soft, delimiter=',')
np.savetxt('gamma_min_p1_iter10_Nce10_f02_soft.csv', gamma_min_p1_iter10_Nce10_f02_soft, delimiter=',')
np.savetxt('gamma_mean_p1_iter10_Nce10_f02_soft.csv', gamma_mean_p1_iter10_Nce10_f02_soft, delimiter=',')
np.savetxt('Q_min_p1_iter10_Nce10_f02_soft.csv', Q_min_p1_iter10_Nce10_f02_soft, delimiter=',')
np.savetxt('Q_mean_p1_iter10_Nce10_f02_soft.csv', Q_mean_p1_iter10_Nce10_f02_soft, delimiter=',')

In [ ]:
# How to load data
data = np.loadtxt('beta_min_p1_iter1_Nce5_f1_soft.csv', delimiter=',')

In [ ]:
## HARD

p=2   #Circuit depth # 1,2,3
iterations=10; # Number of iterations for cross entropy 1,5,10,20
Nce_samples=1000; # Number of samples 10,15,20
f_elite=0.2;   # fraction of best samples 0.2,0.4, 1

Num=1# Number of times we run the loop 
E_treshold=0.005; # Maximum energy which we tolerate
R=1000; # Number of times we average the quality parameters
beta_min=np.zeros((Num,p)) #Circuit parameters beta and gamma for min strategy
gamma_min=np.zeros((Num,p))
beta_mean=np.zeros((Num,p)) #Circuit parameters beta and gamma for mean strategy
gamma_mean=np.zeros((Num,p))
Q_min=np.zeros((Num,3))    # Quality parameters for min strategy
Q_mean=np.zeros((Num,3))   # Quality parameters for mean strategy

for ii in range(Num):
    beta_min[ii],gamma_min[ii]=cross_entropy_min_hard(p,iterations,Nce_samples,f_elite)
    beta_mean[ii],gamma_mean[ii]=cross_entropy_mean_hard(p,iterations,Nce_samples,f_elite)
    Q_min[ii]=quality_parameters_hard(beta_min[ii],gamma_min[ii],E_treshold,R)
    Q_mean[ii]=quality_parameters_hard(beta_mean[ii],gamma_mean[ii],E_treshold,R)
    
beta_min,gamma_min,Q_min, Q_mean

MIN
k=0, mean_E=0.028631, std_E=0.004197, beta=[5.44940465 0.82246851], gamma=[5.22695011 5.98511782]
k=1, mean_E=0.029673, std_E=0.004737, beta=[5.54125153 0.74627375], gamma=[5.2198559  6.05193062]
k=2, mean_E=0.029659, std_E=0.005974, beta=[5.51344079 0.702925  ], gamma=[5.28349972 6.12715572]
k=3, mean_E=0.030580, std_E=0.006367, beta=[5.52417964 0.66575726], gamma=[5.39309724 6.09400596]
k=4, mean_E=0.031598, std_E=0.008375, beta=[5.36708756 0.62187294], gamma=[5.29920642 6.1937121 ]
k=5, mean_E=0.031578, std_E=0.007815, beta=[5.32760244 0.61022472], gamma=[5.22943712 6.30627908]
k=6, mean_E=0.028685, std_E=0.004169, beta=[5.35636427 0.54333304], gamma=[5.16235583 6.3814813 ]
k=7, mean_E=0.030555, std_E=0.007020, beta=[5.41186416 0.59037768], gamma=[5.15550501 6.40539249]
k=8, mean_E=0.033823, std_E=0.012342, beta=[5.4295875  0.60185642], gamma=[5.139249   6.45142063]
k=9, mean_E=0.033349, std_E=0.010007, beta=[5.39560304 0.69234565], gamma=[5.17562187 6.45619175]
MEAN
k=0, mean_E

In [11]:
## HARD
beta_min_p2_iter10_Nce15_f02_hard=beta_min
gamma_min_p2_iter10_Nce15_f02_hard=gamma_min
beta_mean_p2_iter10_Nce15_f02_hard=beta_mean
gamma_mean_p2_iter10_Nce15_f02_hard=gamma_mean
Q_min_p2_iter10_Nce15_f02_hard=Q_min
Q_mean_p2_iter10_Nce15_f02_hard=Q_mean
np.savetxt('beta_min_p2_iter10_Nce15_f02_hard.csv', beta_min_p2_iter10_Nce15_f02_hard, delimiter=',')
np.savetxt('beta_mean_p2_iter10_Nce15_f02_hard.csv', beta_mean_p2_iter10_Nce15_f02_hard, delimiter=',')
np.savetxt('gamma_min_p2_iter10_Nce15_f02_hard.csv', gamma_min_p2_iter10_Nce15_f02_hard, delimiter=',')
np.savetxt('gamma_mean_p2_iter10_Nce15_f02_hard.csv', gamma_mean_p2_iter10_Nce15_f02_hard, delimiter=',')
np.savetxt('Q_min_p2_iter10_Nce15_f02_hard.csv', Q_min_p2_iter10_Nce15_f02_hard, delimiter=',')
np.savetxt('Q_mean_p2_iter10_Nce15_f02_hard.csv', Q_mean_p2_iter10_Nce15_f02_hard, delimiter=',')